# Programming Pearldle for Perpetual Play

One of the most bittersweet moments of getting my hand on the world download was playing the _very final week_ of Pearldle, aka Dye-Duction ~~Especially since I washed and only got it after six guesses 😭~~.

And while Pearl left clear instructions to allow people to set new word for friends and family, that leaves those of us lonely folk who play single-player high-and-dry.

It's too bad that there's no way to set items in a container's inventory programmatically...

## Prep work

In my copy of the world download, I went into Pearldle and wrote down the coordinates of the letter chest and the hoppers that need to be set. They are:

| Container | x | y | z |
| --- | --- | --- | --- |
| Letter Chest | 575 | 58 | 148 |
| Step 2 Letter 1 Hopper (41 items) | 571 | 56 | 158 |
| Step 2 Letter 2 Hopper (41 items) | 571 | 56 | 157 |
| Step 2 Letter 3 Hopper (41 items) | 571 | 56 | 156 |
| Step 2 Letter 4 Hopper (41 items) | 571 | 56 | 155 |
| Step 2 Letter 5 Hopper (41 items) | 571 | 56 | 154 |
| Step 2 Letter 1 Hopper (1 items) | 571 | 55 | 158 |
| Step 2 Letter 2 Hopper (1 items) | 571 | 55 | 157 |
| Step 2 Letter 3 Hopper (1 items) | 571 | 55 | 156 |
| Step 2 Letter 4 Hopper (1 items) | 571 | 55 | 155 |
| Step 2 Letter 5 Hopper (1 items) | 571 | 55 | 154 |
| Step 3 Letter 1 Hopper (41 items) | 556 | 56 | 158 |
| Step 3 Letter 2 Hopper (41 items) | 556 | 56 | 157 |
| Step 3 Letter 3 Hopper (41 items) | 556 | 56 | 156 |
| Step 3 Letter 4 Hopper (41 items) | 556 | 56 | 155 |
| Step 3 Letter 5 Hopper (41 items) | 556 | 56 | 154 |
| Step 3 Letter 1 Hopper (1 items) | 556 | 55 | 158 |
| Step 3 Letter 2 Hopper (1 items) | 556 | 55 | 157 |
| Step 3 Letter 3 Hopper (1 items) | 556 | 55 | 156 |
| Step 3 Letter 4 Hopper (1 items) | 556 | 55 | 155 |
| Step 3 Letter 5 Hopper (1 items) | 556 | 55 | 154 |

I also need a Wordle dictionary. Any old list of five-letter words won't work because it'll contain plurals and scientific terms that are absolutely no fun to play. There are lists of past answers, word lists used by popular wordle clones and even the original list of 2,315 words selected by Josh Wardle. But there are copyright and distribution implications, so... why won't I go with something a little bit more personal? Pearl's very own list of past words?

Those can be culled from the book found in Chat's lectern at `(594, 60, 154)`

Note that:
- Chat lost a few times and didn't have that week's answer recorded
- But Pearl also wouldn't have let chat play an invalid word, so _any guesses_ are fair game.

## Setup and Macros

In [1]:
import copy
import json
import random
from collections import Counter
from collections.abc import Collection
from functools import partial
from os import environ
from pathlib import Path
from typing import Any

import mutf8
import pandas as pd
from IPython.display import Markdown, display
from nbt import nbt, region

In [2]:
def format_file_size(path: Path) -> str:
    """Print the size of the specified file in
    human-readible form (KB / MB / GB)

    Parameters
    ----------
    path : Path
        The path to the file

    Returns
    -------
    str
        A prettily formatted file size

    Notes
    -----
    I would be shocked if there isn't a utility already built
    into the standard library to do this, but all I could find
    via Googling was a bunch of recipes and examples
    """
    size: float = path.stat().st_size  # in bytes
    for unit in ("B", "KB", "MB", "GB"):
        if size < 1024 / 2:
            return f"{size:.1f} {unit}"
        size /= 1024
    return f"{size} TB"

In [3]:
def summarize_keystore(keystore: dict[str, Any]) -> None:
    """Display a summary of the contents of a key-value store

    Parameters
    ----------
    keystore : dict
        The keystore to summarize

    Returns
    -------
    None
    """

    def _summarize_keystore(keystore: dict[str, Any]) -> str:
        summary = ""
        for k, v in keystore.items():
            summary += f"\n - `{k}` : "
            if isinstance(v, (str, nbt.TAG_String)):
                summary += f'`"{v}"`'
            elif not isinstance(v, Collection):
                summary += f"`{str(v)}`"
            else:
                length = len(v)
                if 0 < length < 3:
                    summary += "\n"
                    if not isinstance(v, dict):
                        v = {i: item for i, item in enumerate(v)}
                    summary += "\n".join(
                        (f"\t{line}" for line in _summarize_keystore(v).split("\n"))
                    )
                else:
                    summary += f"({len(v)} items)"
        return summary

    display(Markdown(_summarize_keystore(keystore)))

In [4]:
save_folder = Path(environ["SAVE_PATH"])

# make sure this is set correctly
for path in sorted(save_folder.glob("*")):
    print(f"- {path.name} ({'folder' if path.is_dir() else format_file_size(path)})")

- DIM-1 (folder)
- DIM1 (folder)
- advancements (folder)
- audio_player_data (folder)
- carpet-fixes.conf (22.0 B)
- carpet.conf (57.0 B)
- data (folder)
- datapacks (folder)
- entities (folder)
- icon.png (9.0 KB)
- level.dat (3.3 KB)
- level.dat_old (3.3 KB)
- playerdata (folder)
- poi (folder)
- region (folder)
- resources.zip (34.0 MB)
- scripts (folder)
- session.lock (3.0 B)
- stats (folder)


### Load the region file

Peardle doesn't straddle a region boundary (thank God) so everything is contained within `r.1.0.mca`

In [5]:
region_data = region.RegionFile(save_folder / "region" / "r.1.0.mca")
region_start = (512, 0)

## Let's look at a letter

In [6]:
letter_chest_pos = (575, 58, 148)
chunk = region_data.get_chunk(
    (letter_chest_pos[0] - region_start[0]) // 16,
    (letter_chest_pos[2] - region_start[1]) // 16,
)
for entity in chunk["block_entities"]:
    if (entity["x"].value, entity["y"].value, entity["z"].value) == letter_chest_pos:
        assert entity["id"].value == "minecraft:chest"
        letter_chest = entity
        break
else:
    raise RuntimeError("Could not find letter chest")

summarize_keystore(letter_chest)


 - `z` : `148`
 - `x` : `575`
 - `id` : `"minecraft:chest"`
 - `y` : `58`
 - `Items` : (26 items)
 - `keepPacked` : `0`
 - `components` : (0 items)

Good. We have all 26 letters in this chest.

In [7]:
summarize_keystore(letter_chest["Items"][0])


 - `Slot` : `0`
 - `id` : `"minecraft:orange_dye"`
 - `count` : `6`
 - `components` : (3 items)

In [8]:
summarize_keystore(letter_chest["Items"][0]["components"])


 - `minecraft:custom_name` : `"R"`
 - `minecraft:custom_model_data` : 
	
	 - `0` : `"floats"`
 - `minecraft:custom_data` : 
	
	 - `0` : `"CustomRoleplayData"`

From here, we can build up a map of letters to their slots.

In [9]:
letter_map: dict[str, int] = {}
for letter in letter_chest["Items"]:
    letter_map[letter["components"]["minecraft:custom_name"].value] = letter[
        "Slot"
    ].value
letter_map

{'R': 0,
 'A': 1,
 'C': 2,
 'H': 3,
 'Z': 4,
 'W': 5,
 'Y': 6,
 'M': 7,
 'P': 8,
 'B': 9,
 'Q': 10,
 'F': 11,
 'N': 12,
 'K': 13,
 'L': 14,
 'X': 15,
 'G': 16,
 'S': 17,
 'E': 18,
 'O': 19,
 'T': 20,
 'U': 21,
 'J': 22,
 'D': 23,
 'I': 24,
 'V': 25}

## Let's look at a hopper

***Warning!* Running cells beyond this point bay spoil the world download's Pearldle answer!! Read no further until you've played that game!!**

In [10]:
hopper_pos = (571, 56, 158)
chunk = region_data.get_chunk(
    (hopper_pos[0] - region_start[0]) // 16, (hopper_pos[2] - region_start[1]) // 16
)
for entity in chunk["block_entities"]:
    if (entity["x"].value, entity["y"].value, entity["z"].value) == hopper_pos:
        assert entity["id"].value == "minecraft:hopper"
        hopper = entity
        break
else:
    raise RuntimeError("Could not find hopper")

summarize_keystore(hopper)


 - `z` : `158`
 - `x` : `571`
 - `TransferCooldown` : `0`
 - `id` : `"minecraft:hopper"`
 - `y` : `56`
 - `Items` : (5 items)
 - `keepPacked` : `0`
 - `components` : (0 items)

In [11]:
summarize_keystore(hopper["Items"][0])


 - `Slot` : `0`
 - `id` : `"minecraft:peony"`
 - `count` : `41`
 - `components` : (3 items)

Great! I have the right slot. Just to verify:

In [ ]:
final_pearldle = ""
for hopper_num in range(5):
    hopper_pos = (571, 56, 158 - hopper_num)
    chunk = region_data.get_chunk(
        (hopper_pos[0] - region_start[0]) // 16, (hopper_pos[2] - region_start[1]) // 16
    )
    for entity in chunk["block_entities"]:
        if (entity["x"].value, entity["y"].value, entity["z"].value) == hopper_pos:
            assert (
                entity["id"].value == "minecraft:hopper"
            ), f"Block entity at {hopper_pos} is not a hopper"
            assert (
                entity["Items"][0]["count"].value == 41
            ), f"Block entity at {hopper_pos} does not have 41 items in its first slot"
            hopper = entity
            break
    else:
        raise RuntimeError(f"Could not find hopper {hopper_num + 1}")
    final_pearldle += hopper["Items"][0]["components"]["minecraft:custom_name"].value
final_pearldle

Great! Now let's run this verifier across the other three sets of hoppers

In [13]:
answer = ""
for hopper_num in range(5):
    hopper_pos = (571, 55, 158 - hopper_num)
    chunk = region_data.get_chunk(
        (hopper_pos[0] - region_start[0]) // 16, (hopper_pos[2] - region_start[1]) // 16
    )
    for entity in chunk["block_entities"]:
        if (entity["x"].value, entity["y"].value, entity["z"].value) == hopper_pos:
            assert (
                entity["id"].value == "minecraft:hopper"
            ), f"Block entity at {hopper_pos} is not a hopper"
            assert (
                entity["Items"][0]["count"].value == 1
            ), f"Block entity at {hopper_pos} does not have 1 item in its first slot"
            hopper = entity
            break
    else:
        raise RuntimeError(f"Could not find hopper {hopper_num + 1}")
    answer += hopper["Items"][0]["components"]["minecraft:custom_name"].value
assert answer == final_pearldle

In [14]:
answer = ""
for hopper_num in range(5):
    hopper_pos = (556, 56, 158 - hopper_num)
    chunk = region_data.get_chunk(
        (hopper_pos[0] - region_start[0]) // 16, (hopper_pos[2] - region_start[1]) // 16
    )
    for entity in chunk["block_entities"]:
        if (entity["x"].value, entity["y"].value, entity["z"].value) == hopper_pos:
            assert (
                entity["id"].value == "minecraft:hopper"
            ), f"Block entity at {hopper_pos} is not a hopper"
            assert (
                entity["Items"][0]["count"].value == 41
            ), f"Block entity at {hopper_pos} does not have 41 item in its first slot"
            hopper = entity
            break
    else:
        raise RuntimeError(f"Could not find hopper {hopper_num + 1}")
    answer += hopper["Items"][0]["components"]["minecraft:custom_name"].value
assert answer == final_pearldle

In [15]:
answer = ""
for hopper_num in range(5):
    hopper_pos = (556, 55, 158 - hopper_num)
    chunk = region_data.get_chunk(
        (hopper_pos[0] - region_start[0]) // 16, (hopper_pos[2] - region_start[1]) // 16
    )
    for entity in chunk["block_entities"]:
        if (entity["x"].value, entity["y"].value, entity["z"].value) == hopper_pos:
            assert (
                entity["id"].value == "minecraft:hopper"
            ), f"Block entity at {hopper_pos} is not a hopper"
            assert (
                entity["Items"][0]["count"].value == 1
            ), f"Block entity at {hopper_pos} does not have 1 item in its first slot"
            hopper = entity
            break
    else:
        raise RuntimeError(f"Could not find hopper {hopper_num + 1}")
    answer += hopper["Items"][0]["components"]["minecraft:custom_name"].value
assert answer == final_pearldle

## Setting the word

So now if I wanted to set the word to "PEARL" I could do that by editing the chunk data directly.

In [16]:
for hopper_num, letter in enumerate("PEARL"):
    letter_item = letter_chest["Items"][letter_map[letter]]
    assert letter_item["components"]["minecraft:custom_name"].value == letter
    for hopper_x in (571, 556):
        for hopper_y, item_count in ((56, 41), (55, 1)):
            hopper_pos = (hopper_x, hopper_y, 158 - hopper_num)
            chunk = region_data.get_chunk(
                (hopper_pos[0] - region_start[0]) // 16,
                (hopper_pos[2] - region_start[1]) // 16,
            )
            for entity in chunk["block_entities"]:
                if (
                    entity["x"].value,
                    entity["y"].value,
                    entity["z"].value,
                ) == hopper_pos:
                    assert (
                        entity["id"].value == "minecraft:hopper"
                    ), f"Block entity at {hopper_pos} is not a hopper"
                    assert (
                        entity["Items"][0]["count"].value == item_count
                    ), f"Block entity at {hopper_pos} does not have the correct number of items its first slot"
                    hopper = entity
                    break
            else:
                raise RuntimeError(f"Could not find hopper at {hopper_pos}")
            paste_item = copy.deepcopy(letter_item)
            paste_item["Slot"].value = 0
            paste_item["count"].value = item_count
            hopper["Items"][0] = paste_item
            # writing on each modification is silly and wasteful
            # but it's better than wasting time on caching logic
            region_data.write_chunk(chunk.loc.x, chunk.loc.z, chunk)

## Parsing the Word List

In [17]:
lectern_pos = (594, 60, 154)
chunk = region_data.get_chunk(
    (lectern_pos[0] - region_start[0]) // 16, (lectern_pos[2] - region_start[1]) // 16
)
for entity in chunk["block_entities"]:
    if (entity["x"].value, entity["y"].value, entity["z"].value) == lectern_pos:
        assert entity["id"].value == "minecraft:lectern"
        lectern = entity
        break
else:
    raise RuntimeError("Could not find lectern")

In [18]:
summarize_keystore(lectern)


 - `Page` : `0`
 - `z` : `154`
 - `x` : `594`
 - `Book` : (3 items)
 - `id` : `"minecraft:lectern"`
 - `y` : `60`
 - `keepPacked` : `0`
 - `components` : (0 items)

In [19]:
summarize_keystore(lectern["Book"])


 - `id` : `"minecraft:writable_book"`
 - `count` : `1`
 - `components` : 
	
	 - `0` : `"minecraft:writable_book_content"`

In [20]:
summarize_keystore(lectern["Book"]["components"]["minecraft:writable_book_content"])


 - `pages` : (56 items)

In [21]:
summarize_keystore(
    lectern["Book"]["components"]["minecraft:writable_book_content"]["pages"][0]
)


 - `raw` : `"Phase 60

Guesses: 

- "`

lol, I think that messed with the markdown

In [22]:
repr(
    lectern["Book"]["components"]["minecraft:writable_book_content"]["pages"][0][
        "raw"
    ].value
)

"'Phase 60\\n\\nGuesses: \\n\\n- '"

Onto the next page

In [23]:
repr(
    lectern["Book"]["components"]["minecraft:writable_book_content"]["pages"][1][
        "raw"
    ].value
)

"'Week 1:\\nWon on guess 4/5\\n\\n\\n\\n'"

Didn't record the guesses that week. Too bad.

In [24]:
repr(
    lectern["Book"]["components"]["minecraft:writable_book_content"]["pages"][2][
        "raw"
    ].value
)

"'Week 3:\\n\\nGuesses: 4/5\\n\\n-Adieu\\n-Fauna\\n-Ultra\\n-*Yucca'"

Wow, Pearldle added repeated-letter functionality **way** earlier than I remembered.

### Building up a parser

Let's see how consistent Pearl was

In [25]:
def parse_guesses(page: str) -> list[str]:
    """Parse the valid Pearldle guesses out of
    a page of a book

    Parameters
    ----------
    page : str
        The contents of that page of the book

    Returns
    -------
    list of str
        The five-letter words comprisings
        the guesses on that page
    """
    guesses = []
    for line in page.split("\n"):
        line = line.strip()
        if len(line) == 0:
            continue
        if line.lower().startswith("week"):
            continue
        if line.lower().startswith("guesses"):
            continue
        line = line.replace("-", "").replace("*", "").strip()
        if len(line) == 5:
            guesses.append(line.upper())
    return guesses


parse_guesses(
    lectern["Book"]["components"]["minecraft:writable_book_content"]["pages"][2][
        "raw"
    ].value
)

['ADIEU', 'FAUNA', 'ULTRA', 'YUCCA']

In [26]:
all_guesses: set[str] = set()

for page in lectern["Book"]["components"]["minecraft:writable_book_content"]["pages"][
    2:
]:
    all_guesses.update(parse_guesses(page["raw"].value))
word_list = sorted(all_guesses)
print(f"Parsed {len(word_list)} words")
word_list

Parsed 182 words


['ABIDE',
 'ADIEU',
 'AFTER',
 'ANKLE',
 'ANVIL',
 'AORTA',
 'AUDIO',
 'AWFUL',
 'AWOKE',
 'BACON',
 'BEANS',
 'BERET',
 'BERRY',
 'BINGO',
 'BIRCH',
 'BITES',
 'BLADE',
 'BLAZE',
 'BLUNT',
 'BOARD',
 'BUGGY',
 'CELLO',
 'CHAIN',
 'CHAOS',
 'CHEAT',
 'CHUTE',
 'CLEAN',
 'CLONE',
 'CLUCK',
 'COACH',
 'COMET',
 'COUGH',
 'CRISP',
 'CRUMB',
 'DANCE',
 'DISCO',
 'DOZEN',
 'DRINK',
 'EERIE',
 'EQUIP',
 'ERASE',
 'EXACT',
 'FABLE',
 'FAIRE',
 'FANCY',
 'FAUNA',
 'FEMUR',
 'FINAL',
 'FLAIR',
 'FLUFF',
 'FLUTE',
 'FRAME',
 'GAYER',
 'GHAST',
 'GHOUL',
 'GLEAM',
 'GLUON',
 'GLYPH',
 'GRACE',
 'GRIEF',
 'HEARD',
 'HEART',
 'HIPPO',
 'HOUSE',
 'HYPER',
 'ICONS',
 'IDLED',
 'IMPLY',
 'IRATE',
 'IRONY',
 'JERKY',
 'JIMMY',
 'JOUST',
 'JUICE',
 'JUMBO',
 'KAZOO',
 'KIOSK',
 'KNACK',
 'LARRY',
 'LEMUR',
 'LIGHT',
 'LINES',
 'LIVES',
 'LIVID',
 'LOTUS',
 'LOVED',
 'LUCID',
 'LUNAR',
 'MANGO',
 'MANOR',
 'MATTE',
 'MELON',
 'MERCY',
 'MERGE',
 'METRO',
 'MILKY',
 'MINES',
 'MONEY',
 'MOUTH',
 'MOVES',


One hundred eighty-two words is not bad! I do need to manually remove some plurals, simple conjugations ~~and cheeky guesses~~ _(actually I can't bring myself to remove Larry, and if I'm leaving Larry, I gotta leave Perry)_

In [27]:
for invalid_word in (
    "BEANS",
    "ICONS",
    "IDLED",
    # "LARRY"
    "LINES",
    "LIVES",
    "LOVED",
    "MINES",
    "MOVES",
    "PILED",
    # "PERRY",
):
    word_list.remove(invalid_word)
print(f"{len(word_list)} words remain")

173 words remain


### Setting a New Word

And now we can fully macro-fy the code to set a random Pearldle word

In [28]:
for hopper_num, letter in enumerate(random.choice(word_list)):
    letter_item = letter_chest["Items"][letter_map[letter]]
    assert letter_item["components"]["minecraft:custom_name"].value == letter
    for hopper_x in (571, 556):
        for hopper_y, item_count in ((56, 41), (55, 1)):
            hopper_pos = (hopper_x, hopper_y, 158 - hopper_num)
            chunk = region_data.get_chunk(
                (hopper_pos[0] - region_start[0]) // 16,
                (hopper_pos[2] - region_start[1]) // 16,
            )
            for entity in chunk["block_entities"]:
                if (
                    entity["x"].value,
                    entity["y"].value,
                    entity["z"].value,
                ) == hopper_pos:
                    assert (
                        entity["id"].value == "minecraft:hopper"
                    ), f"Block entity at {hopper_pos} is not a hopper"
                    assert (
                        entity["Items"][0]["count"].value == item_count
                    ), f"Block entity at {hopper_pos} does not have the correct number of items its first slot"
                    hopper = entity
                    break
            else:
                raise RuntimeError(f"Could not find hopper at {hopper_pos}")
            paste_item = copy.deepcopy(letter_item)
            paste_item["Slot"].value = 0
            paste_item["count"].value = item_count
            hopper["Items"][0] = paste_item
            # writing on each modification is silly and wasteful
            # but it's better than wasting time on caching logic
            region_data.write_chunk(chunk.loc.x, chunk.loc.z, chunk)

A word has been set, and I have **no idea** what it could be.

## Shouldn't This Be A Datapack?

So at this point, I've provided anyone who wants with the tools needed to blindly randomize their pearldle. But to run this notebook end-to-end, you'll need a completely clean copy of the world (otherwise, at the very least, a bunch of assertions will fail). I could absolutely turn this into a standalone script—complete with the option to provide your own wordlist—and that would be great! But not everyone has Python preinstalled on their computer or enjoys running commands from the terminal.

What would be _really_ nice is for folks to have a **datapack** that they can drop into their HC10 so they can run a command—say, `/trigger pearldle_randomize`—and have all of this done directly in-game. And I can help with that!

The elements of such a datapack are:
1. Selecting a random number between 1 and the length of the word list (inclusive)
1. Replacing the items in a container's slot with items from a different slot in a different container*
1. Exposing a trigger for players to call

**We could simply set the slot by hand, with components that match all the NBT, but with Minecraft changing how components and custom item data are stored between versions, I'd rather do a strict copy to ensure that the items will in fact stack with each other.*

### Setting the Letters to a Word

We're going to start with the second part first. [The syntax for that](https://minecraft.wiki/w/Commands/item) is:

```mcfunction
item replace block <dstPos> container.0 from block <srcPos> container.<srcSlot>
```

And then we'll need to run an additional command to set the count to the correct value:

```mcfunction
data modify block 571 56 154 Items[{Slot:0b}] merge value {count:<count>}
```

So a script to set the word "PEARL" as the guess might look something like:

In [29]:
for hopper_num, letter in enumerate("PEARL"):
    for hopper_x in (571, 556):
        for hopper_y, item_count in ((56, 41), (55, 1)):
            hopper_pos = (hopper_x, hopper_y, 158 - hopper_num)
            print(
                f"item replace block {hopper_pos[0]} {hopper_pos[1]} {hopper_pos[2]} container.0"
                f" from block {letter_chest_pos[0]} {letter_chest_pos[1]} {letter_chest_pos[2]}"
                f" container.{letter_map[letter]}"
            )
            print(
                f"data modify block {hopper_pos[0]} {hopper_pos[1]} {hopper_pos[2]}"
                " Items[{Slot:0b}] merge value {count:%d}" % item_count
            )

item replace block 571 56 158 container.0 from block 575 58 148 container.8
data modify block 571 56 158 Items[{Slot:0b}] merge value {count:41}
item replace block 571 55 158 container.0 from block 575 58 148 container.8
data modify block 571 55 158 Items[{Slot:0b}] merge value {count:1}
item replace block 556 56 158 container.0 from block 575 58 148 container.8
data modify block 556 56 158 Items[{Slot:0b}] merge value {count:41}
item replace block 556 55 158 container.0 from block 575 58 148 container.8
data modify block 556 55 158 Items[{Slot:0b}] merge value {count:1}
item replace block 571 56 157 container.0 from block 575 58 148 container.18
data modify block 571 56 157 Items[{Slot:0b}] merge value {count:41}
item replace block 571 55 157 container.0 from block 575 58 148 container.18
data modify block 571 55 157 Items[{Slot:0b}] merge value {count:1}
item replace block 556 56 157 container.0 from block 575 58 148 container.18
data modify block 556 56 157 Items[{Slot:0b}] merge va

The only problem with the above is that the game pieces chest _really isn't_ the best place to be snagging letters from for a datapack, since every time you reset the game, letters are going to end up in different spots.

Luckily, there are **barrels** of letters deep in the gubbins that players really shouldn't be messing with unless they're changing the word by hand (and even if they're doing that, they shouldn't be changing the locations of letters in those barrels).

In [30]:
letter_lookup: dict[str, tuple[int, int, int, int]] = {}
barrel_chunk = region_data.get_chunk(4, 10)
for entity in barrel_chunk["block_entities"]:
    if entity["id"].value != "minecraft:barrel":
        continue
    for item in entity["Items"]:
        letter = item.get("components", {}).get("minecraft:custom_name", None)
        if not letter:
            continue
        letter_lookup[letter.value] = (
            entity["x"].value,
            entity["y"].value,
            entity["z"].value,
            item["Slot"].value,
        )
assert len(letter_lookup) == 26, "Looks like we're missing some letters"
letter_lookup

{'E': (579, 55, 168, 19),
 'F': (579, 55, 168, 21),
 'G': (579, 55, 168, 24),
 'H': (579, 55, 168, 26),
 'I': (579, 54, 167, 19),
 'J': (579, 54, 167, 21),
 'K': (579, 54, 167, 24),
 'L': (579, 54, 167, 26),
 'Y': (578, 54, 169, 19),
 'Z': (578, 54, 169, 21),
 'A': (579, 55, 167, 19),
 'B': (579, 55, 167, 21),
 'C': (579, 55, 167, 24),
 'D': (579, 55, 167, 26),
 'U': (577, 55, 169, 19),
 'V': (577, 55, 169, 21),
 'W': (577, 55, 169, 24),
 'X': (577, 55, 169, 26),
 'Q': (578, 55, 169, 19),
 'R': (578, 55, 169, 21),
 'S': (578, 55, 169, 24),
 'T': (578, 55, 169, 26),
 'M': (579, 54, 168, 19),
 'N': (579, 54, 168, 21),
 'O': (579, 54, 168, 24),
 'P': (579, 54, 168, 26)}

Cool, so this is the lookup we'll be using.

In [31]:
def generate_commands_to_set_word(word: str) -> list[str]:
    """Generate a list of commands to set
    Pearldle to a given word.

    Parameters
    ----------
    word: str
        A five-letter word to set as the Pearldle answer

    Returns
    -------
    list of str
        The commands to run to set Pearldle to that word
    """
    assert len(word) == 5, f"{word} is not a five letter word"
    commands: list[str] = []
    for hopper_num, letter in enumerate(word):
        for hopper_x in (571, 556):
            for hopper_y in (56, 55):
                hopper_pos = (hopper_x, hopper_y, 158 - hopper_num)
                commands.append(
                    f"item replace block {hopper_pos[0]} {hopper_pos[1]} {hopper_pos[2]} container.0"
                    f" from block {letter_lookup[letter][0]} {letter_lookup[letter][1]} {letter_lookup[letter][2]}"
                    f" container.{letter_lookup[letter][3]}"
                )
    return commands


display(
    Markdown(
        "```mcfunction\n" + "\n".join(generate_commands_to_set_word("PEARL")) + "\n```"
    )
)

```mcfunction
item replace block 571 56 158 container.0 from block 579 54 168 container.26
item replace block 571 55 158 container.0 from block 579 54 168 container.26
item replace block 556 56 158 container.0 from block 579 54 168 container.26
item replace block 556 55 158 container.0 from block 579 54 168 container.26
item replace block 571 56 157 container.0 from block 579 55 168 container.19
item replace block 571 55 157 container.0 from block 579 55 168 container.19
item replace block 556 56 157 container.0 from block 579 55 168 container.19
item replace block 556 55 157 container.0 from block 579 55 168 container.19
item replace block 571 56 156 container.0 from block 579 55 167 container.19
item replace block 571 55 156 container.0 from block 579 55 167 container.19
item replace block 556 56 156 container.0 from block 579 55 167 container.19
item replace block 556 55 156 container.0 from block 579 55 167 container.19
item replace block 571 56 155 container.0 from block 578 55 169 container.21
item replace block 571 55 155 container.0 from block 578 55 169 container.21
item replace block 556 56 155 container.0 from block 578 55 169 container.21
item replace block 556 55 155 container.0 from block 578 55 169 container.21
item replace block 571 56 154 container.0 from block 579 54 167 container.26
item replace block 571 55 154 container.0 from block 579 54 167 container.26
item replace block 556 56 154 container.0 from block 579 54 167 container.26
item replace block 556 55 154 container.0 from block 579 54 167 container.26
```

I've taken out the count-modifiers, because those need to be set regardless of the word, so there's no sense duplicating those

### Selecting a Random Word

There are two components of this:

1. We'll need to generate a random number and store the result:
   ```mcfunction
   scoreboard players add @s pearlde_guessIndex 0
   execute store result score @s pearlde_guessIndex run random value 1..<numWords>
   ```
2. We'll need to wrap each of the above commands in a conditional check:
   ```mcfunction
   execute if score @s pearldle_guessIndex matches <guessIndex> run ...
   ```

(the `@s`, by the way, can just be the player who triggered the command)

And thus, we should be able to generate the entire massive `mcfunction` file with a simple script:

In [32]:
commands = [
    "scoreboard players add @s pearlde_guessIndex 0",
    f"execute store result score @s pearlde_guessIndex run random value 1..{len(word_list)}",
]
commands.extend([""] * 2)

for i, word in enumerate(word_list):
    commands.append(f"# {word}")
    for command in generate_commands_to_set_word(word):
        commands.append(
            f"execute if score @s pearldle_guessIndex matches {i+1} run {command}"
        )
    commands.append("")

commands.extend([""] * 2)
commands.append("# set proper counts")
for hopper_num in range(5):
    for hopper_x in (571, 556):
        for hopper_y, item_count in ((56, 41), (55, 1)):
            hopper_pos = (hopper_x, hopper_y, 158 - hopper_num)
            commands.append(
                f"data modify block {hopper_pos[0]} {hopper_pos[1]} {hopper_pos[2]}"
                " Items[{Slot:0b}] merge value {count:%d}" % item_count
            )
print(f"Generated {len(commands)} lines of commands")

Generated 3833 lines of commands


That's a big ole file, but from there, but that's it! Aside from that, it should all be cruft.

In [33]:
_ = Path("../../PerpetualPearldle/set_word.mcfunction").write_text(
    "\n".join(commands + [""])
)